In [2]:
import scipy.signal
import numpy as np
import IPython.display as ipd


In [3]:
## Modified from Additive Synthesis by G. Tzanetakis, University of Victoria
## https://github.com/gtzan/synthesizers_cs_perspective/blob/main/src/notebooks/additive_synthcsp.ipynb
def partial(samples, env_tuple, srate, duration):
    nsamples = int(srate*duration)
    data = np.zeros(nsamples)
    env = envelope(data, env_tuple, 
                       srate, duration)
    #plt.plot(env)
    #plt.show()

    np.multiply(samples, env, samples)

## Taken from Additive Synthesis by G. Tzanetakis, University of Victoria
## https://github.com/gtzan/synthesizers_cs_perspective/blob/main/src/notebooks/additive_synthcsp.ipynb
def envelope(data, segments,srate,duration): 
    nsamples = int(srate*duration)
    value = 0.0
    segment_index = 0 
    segment_sample = 0 
    prev_target = 0.0

    for i in np.arange(nsamples): 
        if (segment_index < len(segments)): 
            target = segments[segment_index][0]
            ramp_time = segments[segment_index][1]
            delay_time = segments[segment_index][2]
            
            ramp_samples = (ramp_time / 1000.0) * srate 
            delay_samples = (delay_time / 1000.0) * srate
            
            if i < segment_sample + ramp_samples: 
                incr = (target-prev_target) / ramp_samples 
            elif i < segment_sample + ramp_samples + delay_samples: 
                incr = 0.0 
            else: 
                if ramp_samples != 0.0: 
                    incr = (target-prev_target) / ramp_samples 
                else: 
                    incr = 0.0 
                segment_sample = i 
                segment_index = segment_index+1 
                prev_target = target 
            value = value + incr 
        data[i] = value
    return data

def sinusoid(freq, dur, srate=44100, amp=1.0): 

    data = amp * (np.sin(2 * np.pi * np.arange(srate * dur) * freq / srate)).astype(np.float32)

    return data

srate = 44100 
freq = 150

duration = 3.0

p1 = sinusoid(200 * 1.0, dur=duration)
s1 = [(0.3, 50, 0), (0.1, 100, 0), (0.0, 200, 200)]
partial(p1, s1, srate, duration)
    
p2 = sinusoid(freq * 10.01, dur=duration)
s2 = [(0.5, 50, 0), (0.2, 100, 0), (0.0, 200, 200)]
partial(p2, s2, srate, duration)

p3 = sinusoid(freq * 20.1, dur=duration)
s3 = [(1.0, 50, 0), (0.45, 100, 0), (0.0, 200, 200)]
partial(p3, s3, srate, duration)

p4 = sinusoid(freq * 89.1, dur=duration)
s4 = [(1.0, 50, 0), (0.35, 100, 0), (0.0, 200, 200)]
partial(p4, s4, srate, duration)

coffee_cup1 = (p1+p2+p3+p4)*0.25 
ipd.Audio(coffee_cup1, rate=srate, normalize=False)

In [4]:
p5 = sinusoid(800 * 1.0, dur=duration)
s5 = [(0.1, 50, 0), (0.01, 100, 0), (0.0, 200, 200)]
partial(p5, s5, srate, duration)
    
p6 = sinusoid(freq * 2.75, dur=duration)
s6 = [(1.0, 40, 0), (0.1, 75, 0), (0.0, 200, 200)]
partial(p6, s6, srate, duration)

p7 = sinusoid(freq * 6.4, dur=duration)
s7 = [(0.5, 50, 0), (0.15, 100, 0), (0.0, 200, 200)]
partial(p7, s7, srate, duration)

p8 = sinusoid(freq * 22.4, dur=duration)
s8 = [(1.0, 50, 0), (0.35, 100, 0), (0.0, 200, 200)]
partial(p8, s8, srate, duration)

coffee_cup2 = (p5+p6+p7+p8)*0.25 
ipd.Audio(coffee_cup2, rate=srate, normalize=False)

In [5]:
## Modified from Additive Synthesis by G. Tzanetakis, University of Victoria
## https://github.com/gtzan/synthesizers_cs_perspective/blob/main/src/notebooks/modal_synthesis.ipynb

srate = 48000
N = int(1.5*srate)   # number of samples to compute

impulse = np.zeros(N)
impulse[0] = 1 

def modal_resonance(audio, amp, freq, radius, srate): 
    b = np.ones(1)
    a = np.zeros(3)
    a[0] = 1.0 
    a[1] = -2*radius * np.cos(2*np.pi*freq*(1.0/srate))
    a[2] = radius * radius 
    # apply filter
    filtered_audio = amp * scipy.signal.lfilter(b, a, audio)
    return filtered_audio 
    
def coffee_cup1(fundamental, excitation, N): 
    # Mode parameters
    nModes = 4
    freqs = fundamental * np.array([1.0, 10.01, 20.1, 89.1]) # modal center frequencies
    radii = [ 0.45, 0.9998, 0.999, 0.9999] # modal radii
    amps = [0.4, 0.1, 0.1, 0.08]    
    modes = [] 
    mix = np.zeros(N)
    for m in np.arange(0, nModes): 
        modes.append(modal_resonance(excitation, amps[m], freqs[m], radii[m], srate))
        mix += modes[m]
        # normalize modes after mixing for individual playback 
        modes[m] = 0.5 * (modes[m] / np.max(modes[m]))
    mix = 0.5 * (mix / np.max(mix))
    return mix, modes 

print('N=', N)
print('impulse', impulse.shape)
fundamental = 200
(mix1,modes) = coffee_cup1(fundamental, impulse, N)
ipd.Audio(mix1, rate=srate, normalize=False)


N= 72000
impulse (72000,)


In [6]:
## Modified from Additive Synthesis by G. Tzanetakis, University of Victoria
## https://github.com/gtzan/synthesizers_cs_perspective/blob/main/src/notebooks/modal_synthesis.ipynb

def coffee_cup2(fundamental, excitation, N): 
    # Mode parameters
    nModes = 4
    freqs = fundamental * np.array([1.0, 2.75, 6.4, 22.4]) # modal center frequencies
    radii = [ 0.15, 0.9998, 0.999, 0.99999] # modal radii
    amps = [0.4, 0.1, 0.1, 0.08]    
    modes = [] 
    mix = np.zeros(N)
    for m in np.arange(0, nModes): 
        modes.append(modal_resonance(excitation, amps[m], freqs[m], radii[m], srate))
        mix += modes[m]
        # normalize modes after mixing for individual playback 
        modes[m] = 0.5 * (modes[m] / np.max(modes[m]))
    mix = 0.5 * (mix / np.max(mix))
    return mix, modes 

print('N=', N)
print('impulse', impulse.shape)
fundamental = 800
(mix1,modes) = coffee_cup2(fundamental, impulse, N)
ipd.Audio(mix1, rate=srate, normalize=False)

N= 72000
impulse (72000,)
